This program uses SVC Classification algorithm to detect hand written 0-9 digit.
Program needs single input digit with white background and digit in black color.

Program compares two kernal linear and rbf. It uses GridSearchCV with KFold to identify tuned hyperparameter.

Program detection becomes better after tuning Hyperparameter and RBF performs little better than linear kernal

Program uses Open CV for image processing

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import validation_curve
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

In [ ]:
# Loading training data
train_data = pd.read_csv("train.csv")

In [ ]:
print("Training data shape : ", train_data.shape)
# Data is haivng 42K images and each image is having 785 features including one label
# The image data is fatten from 28*28 image.
# Image is grey scale. Because it doesn't have RGB that is why doesn't have third dimension

In [ ]:
print("----------------------Training data information--------------------- \n")
print(train_data.info())
print("--------------------------------------------------------------------- \n")

In [ ]:
# Lets print how many unique label are there in Training data
print("------------Unique Label-------------")
unique = list(np.sort(train_data.label.unique()))
print(unique)
print("-------------------------------------")
# Output shows 10 labels are there which represent 0-9 digits

In [ ]:
# Lets see how many values are present for each unique label
sns.countplot(x=train_data.label)
# Data is equally distributed, hence it is good training data

In [ ]:
# The below logic show one of the image
# Print 100th image, See we need to take parameters from 1 to end as 0th is label
random_image = train_data.iloc[100, 1:]
print("Shape of data: ", random_image.shape)
#Reshape image so that it can be displayed correctly
random_image = random_image.values.reshape(28,28)
plt.imshow(random_image, cmap='gray')
plt.title(train_data.label[100])

In [ ]:
# Prepare data for training
# Drop label from feature parameter
X = train_data.drop(columns=['label'], axis=1)
y = train_data.label
# Divide data in Train and Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 10)
# Set linear kernal
model_linear = SVC(kernel='linear')
model_linear.fit(X_train, y_train)
# predict
y_pred = model_linear.predict(X_test)

# Print Accuracy
print("accuracy:", accuracy_score(y_true=y_test, y_pred=y_pred), "\n")
# Print Conusion Matrix
print(confusion_matrix(y_true=y_test, y_pred=y_pred))

In [ ]:
# Create SVC with RBF Kernal
non_linear_model = SVC(kernel='rbf')
# fit
non_linear_model.fit(X_train, y_train)
# predict
y_pred = non_linear_model.predict(X_test)

# Print Accuracy
print("accuracy:", accuracy_score(y_true=y_test, y_pred=y_pred), "\n")
# Print Confusion Matrix
print(confusion_matrix(y_true=y_test, y_pred=y_pred))

In [ ]:
# Hyper Parameter Turning
# creating a KFold object with 5 splits 
folds = KFold(n_splits = 5, shuffle = True, random_state = 10)

# Specify range of hyperparameters
# Set the parameters by cross-validation
hyper_params = [ {'gamma': [1e-2, 1e-3, 1e-4],
                     'C': [5,10]}]


# specify model
model = SVC(kernel="rbf")

# set up GridSearchCV()
model_cv = GridSearchCV(estimator = model, 
                        param_grid = hyper_params, 
                        scoring= 'accuracy', 
                        cv = folds, 
                        verbose = 1,
                        return_train_score=True)      

# fit the model
model_cv.fit(X_train, y_train)

# printing the optimal accuracy score and hyperparameters
best_score = model_cv.best_score_
best_hyperparams = model_cv.best_params_

print("The best test score is {0} corresponding to hyperparameters {1}".format(best_score, best_hyperparams))

In [ ]:
# Tune model with identified hyperparameters for RBF kernal
model = SVC(C=10, gamma=0.01, kernel="rbf")

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
# metrics
print("accuracy", accuracy_score(y_test, y_pred), "\n")
print(confusion_matrix(y_test, y_pred), "\n")

In [ ]:
# Lets test how tuned model work on created images
image = cv2.imread("Three.png")
print("Shape of image ", image.shape)
# Convert the image to gray scale as training data is in Gray Scale
image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
print("Shape of Grey Image ", image.shape)

########## Uncomment the below code only when you have white background and black digit
ret, image = cv2.threshold(image, 120, 255, cv2.THRESH_BINARY_INV)
# Normalize the image
image = image/255

# Need to resize image so that it can match the training image size
image = cv2.resize(image, (28,28))

# Lets see how image looks
cv2.imshow("frame",image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Now reshape it to what training data looks
image = np.reshape(image, (-1,784))
print("Shape of new image ", image.shape)


In [ ]:
print("Result of Prediction ", model.predict(image))